# HW #3 Financial Ratio Quantile Strategies
[FINM 33150] Regression Analysis and Quantitative Trading Strategies\
Winter 2022 | Professor Brian Boonstra

_**Due:** Thursday, February 3rd, at 11:00pm\
**Name:** Ashley Tsoi (atsoi, Student ID: 12286230)_

### 1. Fetch and clean data

#### 1-1. Import packages

In [1]:
import quandl
import json
import pandas as pd
pd.set_option("display.precision", 4)
pd.set_option('display.float_format', lambda x: '%.4f' % x)
from pandas.core.common import SettingWithCopyWarning
import warnings
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

import math
import numpy as np
import datetime as dt
import functools
from itertools import permutations

# let plot display in the notebook instead of in a different window
%matplotlib inline 
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = [21, 8]

#### 1-2. Define the functions to fetch data from Quandl

**1-2-1. Get my personal keys** from ../data/APIs.json

In [2]:
f = open('../data/APIs.json')
APIs = json.load(f)
f.close()

**1-2-2. Define date-format helper function**

In [3]:
def assertCorrectDateFormat(date_text):
    try:
        dt.datetime.strptime(date_text, '%Y-%m-%d')
    except ValueError:
        raise ValueError("Incorrect date format, should be YYYY-MM-DD")

**1-2-3. Define function** to retrieve raw data from Quandl

**Documentation:**
```
Zacks Fundamentals Collection B
https://data.nasdaq.com/databases/ZFB/documentation
https://data.nasdaq.com/databases/ZFB/usage/quickstart/python
```

In [4]:
# Define function that retrieves data from Quandl
@functools.lru_cache(maxsize=16) # Cache the function output
def getQuandlZFBData(from_table,secs,start_date,end_date,columns):
    # Get data fom Quandl using quandl.get_table
    # NOTE: missing data for the inputted date will NOT return a row.

    # INPUT         | DATA TYPE                 | DESCRIPTION
    # from_table    | string                    | FC, FR, MT, MKTV, SHRS, or HDM
    # secs          | string / tuple of string  | security name(s)
    # start_date    | string (YYYY-MM-DD)       | start date of data
    # end_date      | string (YYYY-MM-DD)       | end date of data (same as or after start_date)
    # columns       | string / tuple of string  | names of the columns to return
    
    if secs=='all': secs = list(pd.read_csv('../data/zacks-tickers.csv').ticker.unique()) # import all tickers from zacks-tickers

    if type(secs)==str: seclen = 1
    else: seclen=len(secs)
    print("Quandl | START | Retriving Quandl data for {:d} securities from the ZACKS/{} table: \n".format(seclen,from_table))
    
    # Retrieve data using quandl.get_table
    quandl.ApiConfig.api_key = APIs['Quandl']

    if from_table in ['FC','FR','MKTV','SHRS','HDM']:
        data = quandl.get_table('ZACKS/'+from_table,
                                ticker = secs, 
                                per_end_date = {'gte':start_date, 'lte':end_date},
                                qopts = {'columns':list(columns)},
                                paginate = True)
        
        data['per_end_date'] = pd.to_datetime(data['per_end_date'])
        if 'filing_date' in data.columns:
            data['filing_date'] = pd.to_datetime(data['filing_date'])

    elif from_table == 'MT':
        data = quandl.get_table('ZACKS/MT',
                                ticker = secs, 
                                qopts = {'columns':list(columns)},
                                paginate = True)

    else:
        print("from_table is limited to FC, FR, MT, MKTV, SHRS and HDM")
        
    print("Quandl | DONE  | Returning {:d} rows of data from the ZACKS/{} table.\n".format(len(data),from_table))

    return data


@functools.lru_cache(maxsize=16) # Cache the function output
def _getZFBData(secs,start_date,end_date):
    # Merged Zacks data in five tables: FC, FR, MT, MKTV, and SHRS
    # NOTE: missing data for the inputted date will NOT return a row.

    # INPUT         | DATA TYPE                 | DESCRIPTION
    # secs          | string / tuple of string  | security name(s)
    # start_date    | string (YYYY-MM-DD)       | start date of data
    # end_date      | string (YYYY-MM-DD)       | end date of data (same as or after start_date)
    
    # Retrieve data using quandl.get_table
    fc = getQuandlZFBData('FC',secs,start_date,end_date,('ticker','exchange','per_end_date','per_type','zacks_sector_code','basic_net_eps','diluted_net_eps','tot_lterm_debt','net_lterm_debt','filing_date'))
    fr = getQuandlZFBData('FR',secs,start_date,end_date,('ticker','exchange','per_end_date','per_type','ret_invst','tot_debt_tot_equity'))
    mt = getQuandlZFBData('MT',secs,start_date,end_date,('ticker','ticker_type','asset_type'))
    mktv = getQuandlZFBData('MKTV',secs,start_date,end_date,('ticker','per_end_date','per_type','mkt_val'))
    shrs = getQuandlZFBData('SHRS',secs,start_date,end_date,('ticker','per_end_date','per_type','shares_out','avg_d_shares'))

    # Merge the tables
    print("MERGE  | START | \n")

    zacks_1 = fc.merge(fr, how='outer', on=['ticker','exchange','per_end_date','per_type'])
    zacks_2 = mktv.merge(shrs, how='outer', on=['ticker','per_end_date','per_type'])
    zacks_3 = zacks_1.merge(zacks_2, how='outer', on=['ticker','per_end_date','per_type'])
    zacks = zacks_3.merge(mt, how='outer', on='ticker')

    print("MERGE  | DONE  | Returning {:d} rows of ZACKS data.\n".format(len(zacks)))
    
    return zacks


**1-2-4. Define function** to filter / clean raw data

**Requirements:**
```
- US Equities
- not in the automotive, financial or insurance sector
- end-of-day adjusted closing prices are available
- debt/market cap ratio is greater than 0.1
- has feasible calculation of the ratios: 
  - debt to market cap, 
  - return on investment, and 
  - price to earnings. 
  Including for at least one PER END DATE no more than one year old. Debt ratio of zero is OK.
```

In [ ]:
def getCleanZFBData(secs,start_date,end_date):

    # === GET RAW DATA ============================================
    raw_zacks = _getZFBData(secs,start_date,end_date)
    
    # === FILTER ==================================================
    # US Equities only
    zacks = raw_zacks[raw_zacks['exchange'].isin(('NYSE','NASDAQ'))]  # select US stock exchanges
    zacks = zacks[zacks['ticker_type']=='S']                          # S = Securities
    zacks = zacks[zacks['asset_type']=='COM'][zacks.columns]          # COM = Common stocks

    # remove tickers without filing dates (tickers without filing dates are impossible to join on)
    filingDate_filter = zacks[(pd.isull(zacks['filing_date']))]['ticker'].unique()
    zacks = zacks[(~zacks['ticker'].isin(filingDate_filter))]
    
    # not in the automotive, financial or insurance sector
    sector_filter = zacks[(zacks['zacks_sector_code'].isin((5,13)))]['ticker'].unique() # 5 = finance (includes insurance), 13 = Autumotive
    zacks = zacks[(~zacks['ticker'].isin(sector_filter))]
    
    # debt-to-market-cap ratio greater than 0.1 (filter all since we will have enough tickers)
    
    zacks = zacks[(zacks['ticker'].isin(sector_filter))]
    
    


    # === FINAL CLEAN-UP ==========================================
    # drop these columns as they are no longer needed
    zacks.drop(['ticker_type','asset_type','zacks_sector_code'], axis=1, inplace=True)

    zacks

    return zacks[['']]

#### 1-3. Fetch cleaned data using the functions above

**Dates:**
```
January 1, 2014 - January 31, 2021*
```
**Note: fetch data from July 1, 2013 to get all data reported by January 1, 2014*

**1-3-1. Fetch data** 

In [28]:
zacks_fc = getQuandlZFBData('FC','all','2013-07-01','2021-01-31',('ticker','exchange','per_end_date','per_type','zacks_sector_code','basic_net_eps','diluted_net_eps','tot_lterm_debt','net_lterm_debt','filing_date'))
zacks_fr = getQuandlZFBData('FR','all','2013-07-01','2021-01-31',('ticker','exchange','per_end_date','per_type','ret_invst','tot_debt_tot_equity'))
zacks_mt = getQuandlZFBData('MT','all','2013-07-01','2021-01-31',('ticker','ticker_type','asset_type'))
zacks_mktv = getQuandlZFBData('MKTV','all','2013-07-01','2021-01-31',('ticker','per_end_date','per_type','mkt_val'))
zacks_shrs = getQuandlZFBData('SHRS','all','2013-07-01','2021-01-31',('ticker','per_end_date','per_type','shares_out','avg_d_shares'))

Quandl | START | Retriving Quandl data for 8913 securities from the ZACKS/FC table: 

Quandl | DONE  | Returning 191483 rows of data from the ZACKS/FC table.



**1-1-2. Filter data**

In [38]:
zacks_mt.asset_type.unique()

array([None, 'COM', 'ADR', 'CDN', 'MLP', 'ETF', 'CEF'], dtype=object)